In [1]:
import networkx as nx
import network2tikz
import os
from IPython.display import Image
from collections import defaultdict
import numpy as np
from scipy.optimize import minimize, Bounds
from IPython.core.display import display, HTML
from networkx.algorithms import isomorphism
import matplotlib.pyplot as plt
from sage.combinat.symmetric_group_algebra import e as young_symmetrizer
import ipywidgets as widgets
import threading

In [2]:
preparser(False)

In [3]:
mu, nu, rho, sigma, alpha, beta, kappa, theta, zeta, eta = \
            var('mu, nu, rho, sigma, alpha, beta, kappa, theta, zeta, eta')
greek_variables = [mu, nu, rho, sigma, alpha, beta, kappa, theta, zeta, eta]
extended_greek_variables = greek_variables[:]
for i in range(1, 51):
    for l in greek_variables:
        globals()[str(l)+'_'+str(i)] = var(str(l)+'_'+str(i))
        extended_greek_variables.append(globals()[str(l)+'_'+str(i)])
add, floordiv, mul, pow, sub, truediv, add_var, mul_var = \
                        sorted(list(sage.symbolic.operators.arithmetic_operators), key=str)

In [4]:
%%cython
#cython: boundscheck=False, wraparound=False, nonecheck=False, cdivision=True
from libc.math cimport sqrt

# compiled version of the potential to minimise
cpdef double energy(double[:] pos, long[:,:] adj, long n, double edge_length):
    cdef double k
    cdef double e
    cdef double res
    cdef int i
    cdef int j
    
    k = 1.
    e = 0.01
    res = 0
    for i in range(adj.shape[0]):
        res += k*(sqrt((pos[adj[i][0]]-pos[adj[i][1]])**2+(pos[adj[i][0]+n]-pos[adj[i][1]+n])**2)-edge_length)**2
    for i in range(n):
        for j in range(i+1, n):
            if (pos[i]-pos[j])**2+(pos[i+n]-pos[j+n])**2==0:
                return 100
            res += e/sqrt((pos[i]-pos[j])**2+(pos[i+n]-pos[j+n])**2)
    return res

In [5]:
def render(G, debug=False, return_image=False, size=10, edge_length=0.4, out_widget=None, threaded=False):
    """
    Build a tex file from graph G, compile it, transform it into an image and output it to a widget.
    """
    if len(G)==0:
        return
    
#     G = nx.disjoint_union(G, nx.MultiDiGraph()) # reset nodes name, 
    minimum_energy = 10e6
    layout = 'spring_layout'
    fun = lambda pos: energy(pos, np.array(list(G.edges()), dtype=int), len(G), float(edge_length))
    bounds = Bounds(*np.array([(-1, 1)]*(len(G)*2)).T)
    
    x = []
    for j in range(5):
        res = minimize(fun=fun, x0=[np.random.random()*2-1 for i in range(len(G)*2)], bounds=bounds, options={"maxiter": 50})
        y, f = res.x, res.fun
        if f<minimum_energy:
            x = y
            minimum_energy = f
        if debug:
            print("Energy: "+str(f)+", fun eval: "+ str(res.nfev))
            
    fun = lambda th: max([sin(th[0])*x[i]+cos(th[0])*x[i+len(G)] for i in range(len(G))])-\
                     min([sin(th[0])*x[i]+cos(th[0])*x[i+len(G)] for i in range(len(G))])
    
    res = minimize(fun=fun, x0=[float(0)], bounds=Bounds([-float(pi/2)], [float(pi/2)]))
    th = res.x
    if debug:
        print("rotation :"+str(th)+" rad")
    
    layout = {i: (float(cos(th)*x[i]-sin(th)*x[i+len(G)]), float(cos(th)*x[i+len(G)]+sin(th)*x[i])) for i in range(len(G))}
    
    shape_dict = {True: "circle", False: "rectangle"}
    color_dict = {True: "cyan", False: "green"}
    edge_color = {0: "black", 1: "brown", 2: "blue", 3: "purple", -1: "red"}
    visual_style = {}
    visual_style['vertex_shape'] = {n: shape_dict[g] for n, g in nx.get_node_attributes(G,'index').items()}
    visual_style['vertex_color'] = {n: color_dict[g] for n, g in nx.get_node_attributes(G,'index').items()}

    visual_style['layout'] = layout
    visual_style['canvas'] = (float(size), float(size))

    visual_style['edge_label'] = defaultdict(lambda: "")
    for e in G.edges:
        visual_style['edge_label'][e[:2]] += str((G.edges[e]['start_pos'], G.edges[e]['end_pos'])).replace(',', '')
    visual_style['edge_label'] = dict(visual_style['edge_label'])
    visual_style['edge_label_size'] = float(8)

#     visual_style['node_label'] = {n: str(n)+": "+latex(G.nodes[n]['content']) for n in G}

    visual_style['node_label'] = {n: latex(G.nodes[n]['content']) for n in G}
    visual_style['margin'] = float(1.)
    visual_style['edge_not_in_bg'] = False
    visual_style['node_math_mode'] = True
    visual_style['edge_color'] = {}
    for e in G.edges:
        visual_style['edge_color'][e[:2]] = edge_color[G.edges[e]['vector_space']]    
    
    
    name = str(randint(1, 9999999999))
    network2tikz.plot(G, 'tmp_'+name+'.tex', **visual_style)
    
    def work(args):
        out, name = args[0], args[1]
        try:
            os.system("pdflatex -quiet tmp_"+name+".tex")
        except:
            pass
        try:
            os.system("pdfcrop tmp_"+name+".pdf tmp_"+name+"_bis.pdf")
        except:
            pass
        try:
            os.system("pdftoppm tmp_"+name+"_bis.pdf tmp_"+name+" -png")
        except:
            pass
        with out:
            try:
                display(Image("tmp_"+name+"-1.png"))
                sleep(1)
            except:
                pass
        try:
            os.remove("tmp_"+name+".aux")
        except:
            pass
        try:
            os.remove("tmp_"+name+".log")
        except:
            pass
        try:
            os.remove("tmp_"+name+".pdf")
        except:
            pass
        try:
            os.remove("tmp_"+name+"_bis.pdf")
        except:
            pass
        try:
            os.remove("tmp_"+name+".tex")
        except:
            pass
        try:
            os.remove("tmp_"+name+"-1.png")
        except:
            pass
    
    if out_widget is None:
        out_widget = widgets.Output()
    if threaded:
        t = threading.Thread(target=work, args=((out_widget, name),))
        t.start()
    else:
        work((out_widget, name))
    
    return out_widget

In [6]:
class tensor_field:
    def __init__(self, basic_tensors={}, sym={}, forced_in={}, forced_out={}, rules={}):
        """
        Initialize the tensor field.
        """
        self.basic_tensors = basic_tensors # dict of basic tensors
        self.sym = sym # dict of symmetries
        self.rules = rules # dict of simplification rules
        
        vector_spaces = set()
        for a in basic_tensors.values():
            for b, c in a:
                vector_spaces.add(b)
        self.VS = dict(zip(range(len(vector_spaces)), vector_spaces)) # dict of contraction spaces
        self.index_VS = dict(zip(vector_spaces, range(len(vector_spaces)))) # inverse dict of contraction spaces
        
        self.edge_space = {} # contraction spaces in which the edge lives
        for t in self.basic_tensors:
            self.edge_space[t] = sum(([self.index_VS[a]]*b for a, b in self.basic_tensors[t]), [])

        self.fin = {}
        self.fout = {}
        for t in self.basic_tensors:
            self.fin[t]  = forced_in[t]  if t in forced_in  else []
            self.fout[t] = forced_out[t] if t in forced_out else []
        
        self.number_of_indices = {} # number of indices of tensor t
        for t in self.basic_tensors:
            self.number_of_indices[t] = sum(nb_ind for VS, nb_ind in basic_tensors[t])
            
        self.rules_graph_in = {} # tensor object of rule pattern
        self.rules_graph_out = {} # tensor object of rule result
        self.rules_graph_to_match = {} # tensor object, modified for matching 
        self.rules_graph_to_match_line_graph = {} # dual graph of self.rules_graph_to_match
        self.rules_tensor_content = {} # basic tensor content of tensor
        
        for a in self.rules:
            self.rules_graph_to_match[a] = tensor(self, a)
            self.rules_graph_in[a] = tensor(self, a)
            self.rules_graph_out[a] = tensor(self, rules[a])
            
            # remove nodes that are external indices
            self.rules_graph_to_match[a].graph.remove_nodes_from([n for n in self.rules_graph_to_match[a].graph.nodes if
                                                self.rules_graph_to_match[a].graph.nodes[n]['index']])
            self.rules_tensor_content[a] = set([self.rules_graph_in[a].graph.nodes[n]['content']
                                                for n in self.rules_graph_in[a].graph if not self.rules_graph_in[a].graph.nodes[n]['index']])
        ru = self.rules_graph_to_match
        for r in ru:
            ed = ru[r].graph.edges
            ru2 = ru[r].graph.copy()
            for e in list(ed):
#                 if not self.VS[ed[e]['vector_space']][3]: # edge orientation doesn't matter
#                     ru2.add_edge(e[1], e[0], start_pos=ed[e]['end_pos'], end_pos=ed[e]['start_pos'], vector_space=ed[e]['vector_space'])
                ru2.add_edge(e[1], e[0], start_pos=ed[e]['end_pos'], end_pos=ed[e]['start_pos'], vector_space=ed[e]['vector_space']) # always add reverse
            
            for t in set([ru[r].graph.node[n]['content'] for n in ru[r].graph]):
                ru2.add_node(t, content="Anchor_"+str(t), index=False)
            for n in ru[r].graph:
                ru2.add_edge(n, ru[r].graph.node[n]['content'], start_pos=-1, end_pos=-1, vector_space=-1)
#                 ru2.add_edge(ru[r].graph.node[n]['content'], n, start_pos=-1, end_pos=-1, vector_space=-1)
                
            ru[r].graph = ru2.copy()
            self.rules_graph_to_match_line_graph[r] = ru[r].line_graph(ru[r].graph)
        
        
        def penalty_score(r):
            """
            Score for ordering rules. Is it really useful?
            """
            res = 0
            if self.rules[r]==0:
                res -= 10
            res += 4*len(SR(self.rules[r]))
            res += len(self.rules_graph_in[r].graph)
            return res
            
        self.sorted_rules = sorted(self.rules, key=penalty_score)
 
        # generate all permutations based on the given symmetries
        perms = {}
        perms_dict = {}
        for t in sym:
            perms[t] = [] # possible indices permutations for tensor t, with sign change
            indices_sym = [] # indices appearing in user tableaux
            for tableau in sym[t]:
                indices = reduce(union, map(set, tableau), set()) # collect all indices in this tableau
                indices_sym = union(indices_sym, indices) # add it to the current list of indices
            n_tot = sum(nb_ind for VS, nb_ind in basic_tensors[t]) # total number of indices of tensor t
            sym[t] = list(sym[t]) # convert tuple of tableaux to list of tableaux
            for i in set(range(n_tot)).difference(indices_sym):
                sym[t].append([[i]]) # add size 1x1 tableaux for each forgotten index 
            index_tableau = {}
            for i in set(range(n_tot)):
                for j in range(len(sym[t])):
                    for s2 in sym[t][j]:
                        if i in s2:
                            index_tableau[i] = j # records in which young tableau is the index i
            for tableau in sym[t]:
                indices = reduce(union, map(set, tableau), set()) # collect all indices
                mapping = dict(zip(indices, range(1, len(indices)+1))) # records the mapping to [1, 2, 3, ...]
                t2 = [[mapping[n] for n in t3] for t3 in tableau] # replace in sym to get standard young tableau
#                 perms[t].append(dict(young_symmetrizer(t2))) # generate all permutation obtainable from the young tableau and sign
                
                ys = young_symmetrizer(t2)
                SGA = ys.parent()
                dict_of_perms = {}
                # find permutations that let the symetrizer unchanged, or flip the sign
                for perm in Permutations(SGA.n): # There must be a better way to generate 
                    if ys == SGA(perm)*ys:
                        dict_of_perms[tuple(perm)] = 1
                    if ys == -SGA(perm)*ys:
                        dict_of_perms[tuple(perm)] = -1
                perms[t].append(dict_of_perms)
                
            mapping = {}
            reverse_mapping = {}
            for i in range(len(sym[t])):
                indices = reduce(union, map(set, sym[t][i]), set()) # collect all indices
                mapping[i] = dict(zip(indices, range(1, len(indices)+1))) # records the mapping to [1, 2, 3, ...]
                reverse_mapping[i] = dict(zip(range(1, len(indices)+1), indices)) # records the inverse mapping
            perms_dict[t] = {}
            for cp in cartesian_product_iterator(list(map(lambda x: list(x.keys()), perms[t]))): # cp is a tuple of permutations, one per tableau
                perm = [-1]*n_tot
                # honestly I don't remember how the next loop works, but it puts together multiple pieces of permutations in a sigle one
                for i in range(n_tot):
                    perm[i] = reverse_mapping[index_tableau[i]][cp[index_tableau[i]].index(mapping[index_tableau[i]][i])+1] 
                perms_dict[t][tuple(perm)] = product([perms[t][i][cp[i]] for i in range(len(perms[t]))]) # records the perumtation, as well as the sign associated (1 or -1)
        self.index_permutations = perms_dict
        
    def sign_permutation(self, permutation, t):
        """
        Given a complete permutation of indices in dictionnary format, how does it affect the tensor t ?
        - it is equal to t:           return p, 1
        - it is equal to -t:          return p, -1
        - it does something else:     return p, 0

        The permutation can contain holes. In this case the function returns a matching
        (complete permutation, sign) if found, (incomplete permutation, 0) otherwise.
        incomplete permutation contains -1 in place of missing numbers.
        """
        
        if t not in basic_tensors:
            return (0,), 1
        
        p = [-1]*sum(nb_ind for VS, nb_ind in basic_tensors[t])
        for i in permutation:
            p[i] = permutation[i]
        p = tuple(p)
        
        perms = self.index_permutations[t]

        if p in perms:
            return p, perms[p]

        for candidate in perms:
            if all([p[i]==candidate[i] or p[i]==-1 for i in range(len(p))]):
                return candidate, perms[candidate]

        return p, 0
    
    def simplify(self, expr):
        """
        Simplify an expression containing tensors.
        """
        T = tensor(self, expr)
        T.simplify()
        return T.to_expression()
    
    def __call__(self, expr):
        return tensor(self, expr)

In [7]:
class tensor:
    def __init__(self, TF, E=1, graph=None):
        """
        Create a tensor from a symbolic expression or a graph. You should never call this directly.
        """
        if graph is not None:
            self.TF = TF
            self.operator = mul_var
            self.ext_ind = graph.graph['indices']
            self.prefactor = graph.graph['prefactor']
            self.leaf = True
            self.is_scalar = len(graph)==0
            self.operands = [1]
            self.graph = graph.copy()
            self.E = 1
            self.fully_simplified = False
            return
        
        E = SR(E).expand()
        self.TF = TF
        self.operator = E.operator()
        self.ext_ind = []
        
        if self.operator in self.TF.basic_tensors:
            self.prefactor = 1
            self.E = E
            self.leaf = True
            self.is_scalar = False
            self.operands = [E]
            
        elif self.operator is add_var:
            self.prefactor = 1
            self.E = E
            self.leaf = False
            self.operands = [tensor(TF, op) for op in E.operands()]
            self.is_scalar = all([t.is_scalar for t in self.operands])
        
        elif self.operator is pow and E.operands()[1]==2:
            op = [E.operands()[0], E.operands()[0]]
            self.E = 1
            self.prefactor = 1
            for op2 in op:
                if op2.operator() in self.TF.basic_tensors:
                    self.E *= op2
                else:
                    self.prefactor *= op2
            self.leaf = True
            self.is_scalar = self.E.is_trivially_equal(SR(1))
            self.operator = mul_var
            self.operands = op

        elif self.operator is mul_var:
            op = E.operands()
            self.E = 1
            self.prefactor = 1
            self.operands = []
            for op2 in op:
                if op2.operator() in self.TF.basic_tensors:
                    self.E *= op2
                    self.operands += [op2]
                elif op2.operator() is pow and op2.operands()[1]==2:
                    op3 = op2.operands()[0]
                    if op3.operator() in self.TF.basic_tensors:
                        self.E *= op3
                        self.E *= op3
                        self.operands += [op3, op3]
                else:
                    self.prefactor *= op2
            self.leaf = True
            self.is_scalar = self.E.is_trivially_equal(SR(1))
            self.operator = mul_var

        else:
            self.leaf = True
            self.is_scalar = True
            self.E = 1
            self.prefactor = E
            self.operands = [1]

        self.fully_simplified = False
        
        self.generate_graph()
        
    def _latex_(self):
        """
        Return a latex representation of the tensor.
        """
        return SR(self.to_expression())._latex_()
    
    def expand(self):
        """
        Expand the tensor, only useful for sum of sum.
        """
        if self.operator is add_var:
            for op in self.operands:
                op.expand()
            new_op = []
            for i in range(len(self.operands)):
                op = self.operands[i]
                op2 = op.operator
                if op2 is add_var:
                    new_op += op.__copy__().operands
                else:
                    new_op.append(op.__copy__())
            self.operands = new_op
        
    def plot(self, debug=False, size=10, edge_length=0.2, threaded=False):
        """
        Display the tensor in graphical notation.
        """
        if self.leaf:
            return render(self.graph, debug=debug, return_image=False, size=size, edge_length=edge_length, threaded=threaded)
        else:
            out_widget = widgets.Output()
            for op in self.operands:
                render(op.graph, debug=debug, return_image=False, size=size, edge_length=edge_length, out_widget=out_widget, threaded=threaded)
            return out_widget
    
    def graph_leaf(self):
        """
        Transform a leaf of the symbolic expression into a graph.
        """
        if self.is_scalar:
            self.graph = nx.MultiDiGraph(prefactor=self.prefactor, indices=[])
            return
        
        if self.operator in self.TF.basic_tensors:
            res = nx.MultiDiGraph(prefactor=self.prefactor, indices=[])
            res.add_node(int(0), content=self.operator, index=False)
            op = self.E.operands()
            done = [False]*len(op)
            for i in range(len(op)):
                if not done[i]:
                    for j in range(i+1, len(op)):
                        if not done[j]:
                            if op[i]==op[j]:
                                done[i] = True
                                done[j] = True
                                if self.TF.edge_space[self.operator][i]!=self.TF.edge_space[self.operator][j]:
                                    raise Exception("Incompatible index spaces: "+
                                                        str(self.TF.VS[self.TF.edge_space[self.operator][i]])+" and "+
                                                        str(self.TF.VS[self.TF.edge_space[self.operator][j]]))
                                inversion = False
                                if j in self.TF.fout[self.operator] or i in self.TF.fin[self.operator]:
                                    i, j = j, i
                                    inversion = True
                                if j in self.TF.fout[self.operator] or i in self.TF.fin[self.operator]:
                                    raise Exception("Impossible contraction: "+str(op[i]))

                                res.add_edge(0, 0, start_pos=i, end_pos=j,
                                             vector_space=self.TF.edge_space[self.operator][i])
                                if inversion:
                                    i, j = j, i

                                break
                    else:
                        res.add_node(len(res), content=op[i], index=True)
                        if i in self.TF.fout[self.operator]:
                            res.add_edge(0, len(res)-1, start_pos=i, end_pos=0,
                                         vector_space=self.TF.edge_space[self.operator][i])
                        else:
                            res.add_edge(len(res)-1, 0, start_pos=0, end_pos=i,
                                         vector_space=self.TF.edge_space[self.operator][i])
                        res.graph['indices'] += [op[i]]
            self.graph = res
            return
        
        op = self.operands
        if self.operator is pow and op[1]==2:
            op = [op[0], op[0]]
            self.operator = mul_var
        
        if self.operator is not mul_var:
            raise Exception("Error, expected mul_var")
        
        
        indices = [op2.operands() for op2 in op]
        done = [[False]*len(op2.operands()) for op2 in op]
        res = nx.MultiDiGraph(prefactor=self.prefactor, indices=[])
        for i in range(len(op)):
            res.add_node(i, content=op[i].operator(), index=False)
        for i in range(len(op)):
            for k in range(len(indices[i])):
                if not done[i][k]:
                    found = False
                    for j in range(i+1, len(op)):
                        for l in range(len(indices[j])):
                            if not done[j][l] and not done[i][k]:
                                if str(indices[i][k]) == str(indices[j][l]): #PB ????
                                    if self.TF.edge_space[op[i].operator()][k]!=self.TF.edge_space[op[j].operator()][l]:
                                        raise Exception("Incompatible index spaces: "+
                                                        str(self.TF.VS[self.TF.edge_space[op[i].operator()][k]])+" and "+
                                                        str(self.TF.VS[self.TF.edge_space[op[j].operator()][l]]))
                                    inversion = False
                                    if l in self.TF.fout[op[j].operator()] or k in self.TF.fin[op[i].operator()]:
                                        i, j, k, l = j, i, l, k
                                        inversion = True
                                    if l in self.TF.fout[op[j].operator()] or k in self.TF.fin[op[i].operator()]:
                                        raise Exception("Impossible contraction: "+str(indices[i][k]))

                                    res.add_edge(i, j, start_pos=k, end_pos=l, 
                                                 vector_space=self.TF.edge_space[op[i].operator()][k])
                                    if inversion:
                                        i, j, k, l = j, i, l, k
                                    found = True
                                    done[i][k] = True
                                    done[j][l] = True
                    for l in range(k+1, len(indices[i])):
                        if not done[i][l] and not done[i][k]:
                            if indices[i][k]==indices[i][l]:
                                if self.TF.edge_space[op[i].operator()][k]!=self.TF.edge_space[op[i].operator()][l]:
                                        raise Exception("Incompatible index spaces: "+
                                                        str(self.TF.VS[self.TF.edge_space[op[i].operator()][k]])+" and "+
                                                        str(self.TF.VS[self.TF.edge_space[op[i].operator()][l]]))
                                inversion = False
                                if l in self.TF.fout[op[i].operator()] or k in self.TF.fin[op[i].operator()]:
                                    k, l = l, k
                                    inversion = True
                                if l in self.TF.fout[op[i].operator()] or k in self.TF.fin[op[i].operator()]:
                                    raise Exception("Impossible contraction: "+str(indices[i][k]))

                                res.add_edge(i, i, start_pos=k, end_pos=l,
                                             vector_space=self.TF.edge_space[op[i].operator()][k])
                                if inversion:
                                    k, l = l, k
                                found = True
                                done[i][k] = True
                                done[i][l] = True
                    if not found:
                        res.add_node(len(res), content=indices[i][k], index=True)
                        if k in self.TF.fout[op[i].operator()]:
                            res.add_edge(i, len(res)-1, start_pos=k, end_pos=0,
                                         vector_space=self.TF.edge_space[op[i].operator()][k])
                        else:
                            res.add_edge(len(res)-1, i, start_pos=0, end_pos=k,
                                         vector_space=self.TF.edge_space[op[i].operator()][k])
                        res.graph['indices'] += [indices[i][k]]
        self.graph = res
        return
    
    def generate_graph(self):
        """
        Transform a symbolic expression into a graph, or sum of tensor.
        """
        if self.operator is not add_var:
            self.graph_leaf()
            return
        for op in self.operands:
            op.graph_leaf()
        self.graph = nx.MultiDiGraph(prefactor=1, indices=[])
    
    def line_graph(self, G):
        """
        Return the dual graph of G, and propagate the attibutes.
        """
        G2 = nx.line_graph(G)
        nx.set_edge_attributes(G2, {(n1, n2, key): G.nodes[n1[1]] for n1, n2, key in G2.edges})
        nx.set_node_attributes(G2, {(n1, n2, key): G.edges[n1, n2, key] for n1, n2, key in G2.nodes})
        for n1, n2, key in G2.nodes:
            G2.nodes[n1, n2, key]['start_content'] = G.node[n1]['content']
            G2.nodes[n1, n2, key]['end_content']   = G.node[n2]['content']
        return G2

    def share_a_symmetry(self, e1, e2, tensor):
        """
        return True if e1 and e2 appears in the same Young tableau for the specified tensor.
        """
        if e1==e2:
            return True
        if tensor not in self.TF.sym:
            return False
        for tableau in self.TF.sym[tensor]:
            found_e1 = False
            found_e2 = False
            for line in tableau:
                if e1 in line:
                    found_e1 = True
                if e2 in line:
                    found_e2 = True
            if found_e1 and found_e2:
                return True
        return False
    
    def find_step(self):
        """
        Find a rule that can be applied to self by looking for isomorphic subgraph in the dual graph of self.
        """     
        
        # node comparison function
        def test_edge(e1, e2):
            return bool(e1[0]['content'] is e2[0]['content'])
        
        # fast edge comparison function: can accept a false result, mappings need to be checked afterward.
        def test_node(n1, n2):
            if (n1['start_pos']==-1 and n2['start_pos']==-1):
                return n1['end_content']==n2['end_content']
            return (n1['start_pos']==n2['start_pos'] and n1['end_pos']==n2['end_pos']) or \
                        (self.share_a_symmetry(n1['start_pos'], n2['start_pos'], n1['start_content']) and 
                         self.share_a_symmetry(n1['end_pos'], n2['end_pos'], n1['end_content'])) # ensures edges are of the same type

        ed = self.graph.edges
        G2 = self.graph.copy()
        for e in list(ed):
            G2.add_edge(e[1], e[0], start_pos=ed[e]['end_pos'], end_pos=ed[e]['start_pos'], vector_space=ed[e]['vector_space']) # always add reverse
        for t in set([G2.node[n]['content'] for n in G2]):
            G2.add_node(t, content="Anchor_"+str(t), index=False)
        for n in G2:
            if type(n) is int:
                G2.add_edge(n, G2.node[n]['content'], start_pos=-1, end_pos=-1, vector_space=-1)
                G2.add_edge(G2.node[n]['content'], n, start_pos=-1, end_pos=-1, vector_space=-1)
        LG2 = self.line_graph(G2)
        
        rules = self.TF.rules_graph_in
        for r in self.TF.sorted_rules:
            
            # for each rule, look for a mapping
            
            if not set([self.graph.nodes[n]['content'] for n in self.graph if not self.graph.nodes[n]['index']]).issuperset(self.TF.rules_tensor_content[r]):
                continue
            
            LG1 = self.TF.rules_graph_to_match_line_graph[r]
            GM = isomorphism.DiGraphMatcher(LG2, LG1, edge_match=test_edge, node_match=test_node)
            for mapping in GM.subgraph_isomorphisms_iter():
                
                # a mapping candidate was found
                # do we accept it or not ?
                
                edge_map = mapping
                node_map = {}
                for a in edge_map:
                    if type(a[0]) is int:
                        node_map[a[0]] = edge_map[a][0]
                    if type(a[1]) is int:
                        node_map[a[1]] = edge_map[a][1]
                        
                edge_map_2 = {}        
                for e in edge_map: # recover forced ordering
                    if G2.edges[e]['start_pos']!=-1:
                        if G2.edges[e]['end_pos'] in self.TF.fin[G2.nodes[e[1]]['content']] and G2.edges[e]['start_pos'] in self.TF.fout[G2.nodes[e[0]]['content']]:
                            edge_map_2[e] = edge_map[e]
                        elif G2.edges[e]['start_pos'] in self.TF.fin[G2.nodes[e[0]]['content']] and G2.edges[e]['end_pos'] in self.TF.fout[G2.nodes[e[1]]['content']]:
                            edge_map_2[e[1], e[0], 0] = edge_map[e][1], edge_map[e][0], 0
                        else:
                            if e[2]==0: # self loops can cause trouble if not here ???
                                edge_map_2[e] = edge_map[e]
                            

                edge_map = edge_map_2

                edge_map_2 = {}
                for e in edge_map:
                    if all([type(i) is int for i in e]):
                        if not e in self.graph.edges:
                            edge_map_2[e[1], e[0], e[2]] = edge_map[e]
                        else:
                            edge_map_2[e] = edge_map[e]
                
                accept = True

                for n in node_map:
                    perm = {}
                    for e in edge_map_2:
                        if n==e[0]:
                            perm[self.graph.edges[e]['start_pos']] = self.TF.rules_graph_to_match[r].graph.edges[edge_map_2[e]]['start_pos']
                        if n==e[1]:
                            perm[self.graph.edges[e]['end_pos']] = self.TF.rules_graph_to_match[r].graph.edges[edge_map_2[e]]['end_pos']
                    p, s = self.TF.sign_permutation(perm, self.graph.nodes[n]['content'])
                    
                    if s==0:
                        accept = False # symmetry is not a factor of 1 or -1
                        
                if accept:
                    return r, edge_map, node_map # mapping is a real subgraph isomorphism, interrupt the search

        return False, False, False
    
    def normalize_graph(self):
        """
        Rename the nodes of the graph so that the indices are at the end.
        """
        relabel = {}
        for n in self.graph.nodes:
            if self.graph.nodes[n]['index']:
                relabel[n] = n+len(self.graph)
            else:
                relabel[n] = n
        nx.relabel_nodes(self.graph, relabel, copy=False)
        self.graph = nx.relabel.convert_node_labels_to_integers(self.graph, first_label=0, ordering="sorted")
    
    def simplify_metric(self):
        """
        Simplify identity.
        """
        for n in self.graph.nodes:
            for vs in self.TF.VS:
                if self.TF.VS[vs][2] is self.graph.nodes[n]['content']:
                    a = list(self.graph.in_edges(n)) # incoming
                    b = list(self.graph.out_edges(n)) # outgoing

                    if len(a)==2:
                        if not (self.graph.nodes[a[0][0]]['index'] and self.graph.nodes[a[1][0]]['index']):
                            if not (a[0][0], a[0][1], 1) in self.graph.edges: # 2 different incoming edges
                                pos = self.graph.edges[a[0][0], a[0][1], 0]['start_pos'], self.graph.edges[a[1][0], a[1][1], 0]['start_pos']
                                self.graph.remove_node(n)
                                self.graph.add_edge(a[0][0], a[1][0], start_pos=pos[0], end_pos=pos[1], vector_space=vs)
                                self.normalize_graph()
                                return True
                            else: # 2 incoming edges from one same node
                                pos = self.graph.edges[a[0][0], a[0][1], 0]['start_pos'], self.graph.edges[a[0][0], a[0][1], 1]['start_pos']
                                self.graph.remove_node(n)
                                self.graph.add_edge(a[0][0], a[1][0], start_pos=pos[0], end_pos=pos[1], vector_space=vs)
                                self.normalize_graph()
                                return True                

                    if len(b)==2:
                        if not (self.graph.nodes[b[0][1]]['index'] and self.graph.nodes[b[1][1]]['index']):
                            if not (b[0][0], b[0][1], 1) in self.graph.edges: # 2 different outgoing edges
                                pos = self.graph.edges[b[0][0], b[0][1], 0]['end_pos'], self.graph.edges[b[1][0], b[1][1], 0]['end_pos']
                                self.graph.remove_node(n)
                                self.graph.add_edge(b[0][1], b[1][1], start_pos=pos[0], end_pos=pos[1], vector_space=vs)
                                self.normalize_graph()
                                return True

                            else: # 2 outgoing edges from the same node
                                pos = self.graph.edges[b[0][0], b[0][1], 0]['end_pos'], self.graph.edges[b[0][0], b[0][1], 1]['end_pos']
                                self.graph.remove_node(n)
                                self.graph.add_edge(b[0][1], b[1][1], start_pos=pos[0], end_pos=pos[1], vector_space=vs)
                                self.normalize_graph()
                                return True

                    if len(a)==1 and len(b)==1:
                        if a[0][0]==a[0][1]: # isolated self loop
                            self.graph.remove_node(n)
                            self.graph.graph['prefactor'] = self.graph.graph['prefactor']*self.TF.VS[vs][1] # trace(g) = dim(E)
                            self.normalize_graph()
                            return True

                        # one incoming and one outgoing edge: must preserve orientation
                        if not (self.graph.nodes[b[0][1]]['index'] and self.graph.nodes[a[0][0]]['index']):
                            pos = self.graph.edges[a[0][0], a[0][1], 0]['start_pos'], self.graph.edges[b[0][0], b[0][1], 0]['end_pos'], self.graph.nodes[n]['content']
                            self.graph.remove_node(n)
                            self.graph.add_edge(a[0][0], b[0][1], start_pos=pos[0], end_pos=pos[1], vector_space=vs)
                            self.normalize_graph()
                            return True
        return False

    
    def match_step(self):
        """
        Find a rule to apply, then replace the pattern with the corresponding result.
        This is graph surgery, no pun intended.
        
        Return True if something was changes, False otherwise.
        """
        while self.simplify_metric():
            pass
        
        rule, edge_map, node_map = self.find_step()
        
        if rule is False:
            return False
        
        edge_map_2 = {}
        for e in edge_map:
            if not e in self.graph.edges:
                edge_map_2[e[1], e[0], e[2]] = edge_map[e]
            else:
                edge_map_2[e] = edge_map[e]

        sign = []
        index_permutation = {}
        for n in node_map:
            perm = {}
            for e in edge_map_2:
                if n==e[0]:
                    perm[self.graph.edges[e]['start_pos']] = self.TF.rules_graph_to_match[rule].graph.edges[edge_map_2[e]]['start_pos']
                if n==e[1]:
                    perm[self.graph.edges[e]['end_pos']] = self.TF.rules_graph_to_match[rule].graph.edges[edge_map_2[e]]['end_pos']
            p, s = self.TF.sign_permutation(perm, self.graph.nodes[n]['content'])
            sign += [s]
            index_permutation[n] = p
        sign = product(sign)

        
        inverse_node_map = {v: k for k, v in node_map.items()}
        
        external_leg_in = {} # where on the pattern (to match) does the leg connect
        external_leg_out = {}
        G = self.TF.rules_graph_in[rule].graph
        for n in G.nodes:
            if(G.nodes[n]['index']):
                for _, a in G.out_edges(n):
                    external_leg_in[G.nodes[n]['content']] = (a, G.edges[(n, a, 0)]['end_pos']) # (node, pos)
                for a, _ in G.in_edges(n):
                    external_leg_out[G.nodes[n]['content']] = (a, G.edges[(a, n, 0)]['start_pos'])
        
        external_map_in = {} # IN according to the rules, NOT the graph, where does the leg connect on the outside world in T
        for index in external_leg_in:
            for a, b in self.graph.in_edges(inverse_node_map[external_leg_in[index][0]]): 
                for i in range(self.TF.number_of_indices[self.graph.nodes[inverse_node_map[external_leg_in[index][0]]]['content']]):
                    if (a, b, i) in self.graph.edges and self.graph.edges[a, b, i]['end_pos']==external_leg_in[index][1]:
                        external_map_in[index] = (a, self.graph.edges[a, b, i]['start_pos']) # (node, po)
            for b, a in self.graph.out_edges(inverse_node_map[external_leg_in[index][0]]): # case where arrow is reversed
                for i in range(self.TF.number_of_indices[self.graph.nodes[inverse_node_map[external_leg_in[index][0]]]['content']]):
                    if (b, a, i) in self.graph.edges and self.graph.edges[b, a, i]['start_pos']==external_leg_in[index][1]:
                        external_map_in[index] = (a, self.graph.edges[b, a, i]['end_pos'])
        external_map_out = {} # OUT according to the rules, NOT the graph
        for index in external_leg_out:
            for a, b in self.graph.in_edges(inverse_node_map[external_leg_out[index][0]]): # case where arrow is reversed
                for i in range(self.TF.number_of_indices[self.graph.nodes[inverse_node_map[external_leg_out[index][0]]]['content']]):
                    if (a, b, i) in self.graph.edges and self.graph.edges[a, b, i]['end_pos']==external_leg_out[index][1]:
                        external_map_out[index] = (a, self.graph.edges[a, b, i]['start_pos']) # (node, po)
            for b, a in self.graph.out_edges(inverse_node_map[external_leg_out[index][0]]): 
                for i in range(self.TF.number_of_indices[self.graph.nodes[inverse_node_map[external_leg_out[index][0]]]['content']]):
                    if (b, a, i) in self.graph.edges and self.graph.edges[b, a, i]['start_pos']==external_leg_out[index][1]:
                        external_map_out[index] = (a, self.graph.edges[b, a, i]['end_pos'])
        
        self.graph.remove_nodes_from(node_map)
        
        shift = len(self.graph)
                
        
        list_of_replacement = []
        
        
        if self.TF.rules_graph_out[rule].operator is add_var:  
            for op in self.TF.rules_graph_out[rule].operands:
                list_of_replacement.append(op.graph.copy())
             
        else:
            if self.TF.rules_graph_out[rule].prefactor==0: # zero, returning now
                self.graph = nx.MultiDiGraph(prefactor=0)
                self.prefactor = 0
                return True            
            
            G2 = self.TF.rules_graph_out[rule].graph.copy()
            list_of_replacement.append(G2)
        
        
        resulting_tensor = []
        for G2 in list_of_replacement: # START LOOP -------------------------------------------------------------------------------------------------------
            
            pref = G2.graph['prefactor']*sign*self.graph.graph['prefactor']
            
            external_leg_replace_in = {} # where does the replacing pattern connect to the tensor ?
            external_leg_replace_out = {}
            for n in G2.nodes:
                if(G2.nodes[n]['index']):
                    for _, a in G2.out_edges(n):
                        external_leg_replace_in[G2.nodes[n]['content']] = (a, G2.edges[(n, a, 0)]['end_pos']) # (node, pos)
                    for a, _ in G2.in_edges(n):
                        external_leg_replace_out[G2.nodes[n]['content']] = (a, G2.edges[(a, n, 0)]['start_pos'])

            G2.remove_nodes_from([n for n in G2.nodes if G2.nodes[n]['index']])
            
            
            temp_graph = self.graph.copy()
            old_order = list(temp_graph.nodes)
            temp_graph = nx.relabel.convert_node_labels_to_integers(temp_graph, first_label=0, ordering="sorted")
            new_order = list(temp_graph.nodes)
            relabel_mapping = dict(zip(old_order, new_order))

            relabel = {}
            for n in G2.nodes:
                relabel[n] = n+shift
            G2 = nx.relabel_nodes(G2, relabel)
            
#             print(self)
#             print(temp_graph.nodes, G2.nodes, shift)
            temp_graph = nx.union(temp_graph, G2) # the right thing to do

            G3 = temp_graph

            internal_mapping = set()
            external_map_in2 = {}
            external_map_out2 = {}
            for ind, (a, b) in external_map_in.items():
                if a in relabel_mapping:
                    external_map_in2[ind] = relabel_mapping[a], b
                else:
                    for other_ind, (a2, b2) in external_leg_in.items():
                        if other_ind is not ind:
                            if inverse_node_map[a2]==a and external_map_in[other_ind][0]==inverse_node_map[external_leg_in[ind][0]]:
                                if (other_ind, ind) not in internal_mapping:
                                    internal_mapping.add((ind, other_ind)) # arbitrary ordering

                    for other_ind, (a2, b2) in external_leg_out.items():
                        if other_ind is not ind:
                            if inverse_node_map[a2]==a and external_map_out[other_ind][0]==inverse_node_map[external_leg_in[ind][0]]:
                                if (other_ind, ind) not in internal_mapping:
                                    internal_mapping.add((ind, other_ind)) # ordering ???
            for ind, (a, b) in external_map_out.items():
                if a in relabel_mapping:
                    external_map_out2[ind] = relabel_mapping[a], b
                else:
                    for other_ind, (a2, b2) in external_leg_out.items():
                        if other_ind is not ind:
                            if inverse_node_map[a2]==a and external_map_out[other_ind][0]==inverse_node_map[external_leg_out[ind][0]]:
                                if (other_ind, ind) not in internal_mapping:
                                    internal_mapping.add((other_ind, ind)) # arbitrary ordering 
                    for other_ind, (a2, b2) in external_leg_in.items():
                        if other_ind is not ind:
                            if inverse_node_map[a2]==a and external_map_in[other_ind][0]==inverse_node_map[external_leg_out[ind][0]]:
                                if (other_ind, ind) not in internal_mapping:
                                    internal_mapping.add((other_ind, ind)) # ordering ???         

            for index in external_map_in2:
                vs = self.TF.edge_space[G2.node[external_leg_replace_in[index][0]+shift]['content']][external_leg_replace_in[index][1]]
                G3.add_edge(external_map_in2[index][0], external_leg_replace_in[index][0]+shift, start_pos=external_map_in2[index][1],
                            end_pos=external_leg_replace_in[index][1], vector_space=vs)
            for index in external_map_out2:
                vs = self.TF.edge_space[G2.node[external_leg_replace_out[index][0]+shift]['content']][external_leg_replace_out[index][1]]
                G3.add_edge(external_leg_replace_out[index][0]+shift, external_map_out2[index][0], end_pos=external_map_out2[index][1],
                            start_pos=external_leg_replace_out[index][1], vector_space=vs)


    #         print(internal_mapping)
    #         print(external_leg_in)
    #         print(external_leg_out)
            for ind1, ind2 in internal_mapping:
                if ind1 in external_leg_in:
                    vs = self.TF.edge_space[G2.node[external_leg_replace_in[ind1][0]+shift]['content']][external_leg_replace_in[ind1][1]]       
                    if ind2 in external_leg_in:
                        G3.add_edge(external_leg_replace_in[ind1][0]+shift, external_leg_replace_in[ind2][0]+shift,
                            start_pos=external_leg_replace_in[ind1][1],
                            end_pos  =external_leg_replace_in[ind2][1], vector_space=vs)
                    else:
                        G3.add_edge(external_leg_replace_out[ind2][0]+shift, external_leg_replace_in[ind1][0]+shift,
                            start_pos=external_leg_replace_out[ind2][1],
                            end_pos  =external_leg_replace_in[ind1][1], vector_space=vs)
                else:
                    vs = self.TF.edge_space[G2.node[external_leg_replace_out[ind1][0]+shift]['content']][external_leg_replace_out[ind1][1]]
                    if ind2 in external_leg_in:
                        G3.add_edge(external_leg_replace_out[ind1][0]+shift, external_leg_replace_in[ind2][0]+shift,
                            start_pos=external_leg_replace_out[ind1][1],
                            end_pos  =external_leg_replace_in[ind2][1], vector_space=vs)
                    else:
                        G3.add_edge(external_leg_replace_out[ind1][0]+shift, external_leg_replace_out[ind2][0]+shift,
                            start_pos=external_leg_replace_out[ind1][1],
                            end_pos  =external_leg_replace_out[ind2][1], vector_space=vs)



            relabel = {}
            for n in G3.nodes:
                if G3.nodes[n]['index']:
                    relabel[n] = n+len(G3)
                else:
                    relabel[n] = n
            G3 = nx.relabel_nodes(G3, relabel)
            G3 = nx.relabel.convert_node_labels_to_integers(G3, first_label=0, ordering="sorted")
            
            G3.graph['prefactor'] = pref
            
            resulting_tensor.append(tensor(self.TF, graph=G3))
        
        
        if len(resulting_tensor)==1:
            self.graph = resulting_tensor[0].graph
            return True
        else:
            self.leaf = False
            self.graph = nx.MultiDiGraph(prefactor=1, indices=[])
            self.operator = add_var
            self.prefactor = 1
            self.operands = resulting_tensor
            return True
        
    def simplify_step(self):
        """
        Apply a single simplification step.
        Return True if somthing was changed, False otherwise.
        """
        if self.leaf:
            if self.fully_simplified:
                return False
            if self.match_step():
                return True
            else:
                self.fully_simplified = True
                return False
        
        res = []
        for op in self.operands:
            if not op.fully_simplified:
                if op.match_step():
                    res.append(True)
                else:
                    op.fully_simplified = True
                    res.append(False)
            else:
                res.append(False)
                
        self.expand()
        self.remove_zero_contributions()
        
        if res.count(False)/len(res) > 0.9:
            self.simplify_isom()
        
        return any(res)
            
    def simplify(self):
        """
        Apply simplification steps for as long as needed.
        """
        while self.simplify_step():
            pass
    
    def remove_zero_contributions(self):
        """
        Remove tensors equal to zero in a sum
        """
        if self.operator is not add_var:
            return
        
        n = len(self.operands)
        for i, op in enumerate(reversed(self.operands)):
            if SR(op.prefactor).is_trivial_zero():
                del self.operands[n-i-1]
        
        
#         self.operands = [op for op in self.operands if op.prefactor!=0]
    
    def __add__(self, other):
        """
        Add 2 tensors. Should be improved
        """
        if self.operator is not add_var and other.operator is not add_var:
            res = tensor(self.TF, 1)
            res.operator = add_var
            res.operands = [self.__copy__(), other.__copy__()]
            res.leaf = False
            res.E = self.E+other.E
            res.is_scalar = self.is_scalar and other.is_scalar   
            res.prefactor = 1
            return res

        if self.operator is add_var and other.operator is not add_var:
            res = self.__copy__()
            res.operands.append(other.__copy__())
            res.E += other.E
            return res

        if self.operator is not add_var and other.operator is add_var:
            res = other.__copy__()
            res.operands.append(self.__copy__())
            res.E += self.E
            return res

        if self.operator is add_var and other.operator is add_var:
            res = other.__copy__()
            res.operands += self.__copy__().operands
            res.E += self.E
            return res
 
    def __copy__(self):
        """
        Shallow copy. 
        """
        res = tensor(self.TF, 1)
        res.operator = self.operator
        if res.operator is add_var:
            res.operands = [op.__copy__() for op in self.operands]
        else:
            res.operands = copy(self.operands)
        res.leaf = self.leaf
        res.E = self.E
        res.is_scalar = self.is_scalar 
        res.prefactor = self.prefactor
        res.graph = self.graph.copy()
        res.fully_simplified = self.fully_simplified
        return res
    
    def __mul__(self, other):
        """
        Multiply 2 tensors. Should be improved
        """
        if self.operator is not add_var and other.operator is not add_var:
            res = tensor(self.TF, 1)
            res.operator = mul_var
            res.operands = []
            res.leaf = True
            res.ext_ind = self.ext_ind+other.ext_ind
            res. E = self.E*other.E
            res.is_scalar = self.is_scalar and other.is_scalar
            G = other.graph.copy()
            rename = {}
            for n in G.nodes:
                rename[n] = n+len(self.graph)
            nx.relabel_nodes(G, rename, copy=False)
            res.graph = nx.union(self.graph, G)
            res.normalize_graph()
            res.graph.graph['prefactor'] = self.prefactor*other.prefactor
            res.prefactor = self.prefactor*other.prefactor
            res.graph.graph['indices'] = res.ext_ind
            return res

        if self.operator is add_var and other.operator is not add_var:
            res = []
            for op in self.operands:
                res.append(op*other)
            return sum(res[1:len(res)], res[0])

        if self.operator is not add_var and other.operator is add_var:
            return other.__mul__(self)

        if self.operator is add_var and other.operator is add_var:
            res = []
            for op in self.operands:
                for op2 in other.operands:
                    res.append(op*op2)
            return sum(res[1:len(res)], res[0])
        
    def simplify_isom(self):
        """
        Look for isomorphisms between terms in a sum of tensors, to factorize de contributions.
        This function can slow because if n is the number of non_isomorphic terms, it will perform
        at least n(n-1)/2 isomorphism tests.
        """
        
        if self.operator is not add_var:
            return

        def test_edge(e1, e2):
        #     print(e1, e2)
            return bool(str(e1[0]['content'])==str(e2[0]['content']))

        def test_node(n1, n2):
        #     print(n1, n2)
            return (n1['start_pos']==n2['start_pos'] and n1['end_pos']==n2['end_pos']) or \
                        (self.share_a_symmetry(n1['start_pos'], n2['start_pos'], n1['start_content']) and 
                         self.share_a_symmetry(n1['end_pos'], n2['end_pos'], n1['end_content'])) # ensures edges are of the same type

        classes = {}
        representant = {}
        line_graphs = {}
        for i in range(len(self.operands)):
            G1 = self.operands[i].graph.copy()
            G10 = self.operands[i].graph.copy()
            ed = G1.edges
            for e in list(ed):
                G1.add_edge(e[1], e[0], start_pos=ed[e]['end_pos'], end_pos=ed[e]['start_pos'], vector_space=ed[e]['vector_space'])
            for e in ed:
                G1.edges[e]['start_content'] = G1.node[e[0]]['content']
                G1.edges[e]['end_content'] = G1.node[e[1]]['content']
            line_graphs[i] = self.line_graph(G1)
            for j in representant:
                G2 = representant[j]
                accepted = False
                if isomorphism.faster_could_be_isomorphic(line_graphs[i], line_graphs[j]): # 20% speedup
                    GM = isomorphism.DiGraphMatcher(line_graphs[i], line_graphs[j], edge_match=test_edge, node_match=test_node)
                    for mapping in GM.isomorphisms_iter():
                        edge_map = mapping
                        node_map = {}
                        for a in edge_map:
                            node_map[a[0]] = edge_map[a][0]
                            node_map[a[1]] = edge_map[a][1]

                        edge_map_2 = {}
                        for e in edge_map:
                            if e in G10.edges and all([G1.edges[e][a]==G10.edges[e][a] for a in G10.edges[e]]):
                                edge_map_2[e] = edge_map[e]

    #                     print(all([e in G2.edges for e in edge_map.values()]), all([e in G1.edges for e in edge_map.values()]))

                        accept = True
                        sign = []
                        for n in node_map:
                            perm = {}
                            for e in edge_map_2:

    #                             print(G10.edges[e])
    #                             print(G2.edges[edge_map_2[e]])

                                if n==e[0]:
                                    perm[G10.edges[e]['start_pos']] = G2.edges[edge_map_2[e]]['start_pos']
                                if n==e[1]:
                                    perm[G10.edges[e]['end_pos']] = G2.edges[edge_map_2[e]]['end_pos']
                            p2, s = self.TF.sign_permutation(perm, G1.nodes[n]['content'])
                            sign += [s]

                            if s==0:
                                accept = False

                        sign = product(sign)
                        if accept:
                            classes[j] += sign*G1.graph['prefactor']
                            accepted = True
                            break

                    if accepted:
                        break               

            else:
                classes[i] = G1.graph['prefactor']
                representant[i] = G1
                

#         print(len(self.operands), len(classes))
        for i in classes:
            self.operands[i].prefactor = classes[i]
            self.operands[i].graph.graph['prefactor'] = classes[i]

        self.operands = [self.operands[i] for i in classes]

        if len(self.operands)==1:
            self.operator = mul_var
            self.graph = self.operands[0].graph
            self.prefactor = self.graph.graph['prefactor']
            self.leaf = True
            self.operands = [1]
            
    def to_expression(self):
        """
        Convert graph to symbolic expression.
        """
        
        if self.operator is add_var and len(self.operands)>1:
            return sum([op.to_expression() for op in self.operands])


        def next_ind(i=-1, veto=[]):
            i += 1
            while(extended_greek_variables[i] in veto):
                i += 1
            return i
        
        indices = []
        for n in self.graph.nodes:
            if self.graph.nodes[n]['index']:
                indices.append(self.graph.nodes[n]['content'])
                
        edge_map = {}
        ind = next_ind(veto=indices)
        for e in self.graph.edges:
            if not self.graph.nodes[e[0]]['index'] and not self.graph.nodes[e[1]]['index']:
                edge_map[e] = extended_greek_variables[ind]
                ind = next_ind(ind, veto=indices)
            elif self.graph.nodes[e[0]]['index']:
                edge_map[e] = self.graph.nodes[e[0]]['content']
            elif self.graph.nodes[e[1]]['index']:
                edge_map[e] = self.graph.nodes[e[1]]['content']
            else:
                raise Exception("Two external indices are contracted with each other, this is not supposed to happen.")     
        res = self.graph.graph['prefactor']
        
        for n in self.graph.nodes: 
            if not self.graph.nodes[n]['index']:
                current_indices = list(self.graph.out_edges(n, keys=True))+list(self.graph.in_edges(n, keys=True))
                order = [self.graph.edges[e]['start_pos'] for e in self.graph.out_edges(n, keys=True)]+[self.graph.edges[e]['end_pos'] for e in self.graph.in_edges(n, keys=True)]
                res *= self.graph.nodes[n]['content'](*[edge_map[current_indices[order.index(i)]] for i in range(len(order))])
        return res
    

In [8]:
preparser(True)